In [2]:
import sys

# add project root directory to python path to enable import of saapy
if ".." not in sys.path:
    sys.path.append('..')
    
import logging
logging.captureWarnings(True) # mostly warnings caused by self-signed certs

from saapy import Workspace, Environment

from collections import namedtuple

def setup_env():
    ws = Workspace.from_home("iDempiere", rel_root_dir="Projects", work_dir="iDempiere")
    env = Environment(ws)
    env.setup()
    return env

env = setup_env()

In [4]:
def find_reporting_debt():
    query = """
    MATCH 
        (file:JavaFile)-[:DEFINES]->
        (zk_coupled_class:JavaClass)
        -[zk_couple:COUPLES]->
        (zk_class:JavaClass) 
    WHERE 
        zk_class.name =~ {zk_class_name_exp} AND 
        zk_coupled_class.name =~ {class_name_exp}
    WITH file, zk_coupled_class, zk_couple
    MATCH 
        (file:JavaFile)-[:DEFINES]->
        (zk_coupled_class:JavaClass)
        -[db_couple:COUPLES]->
        (db_class:JavaClass) 
    WHERE db_class.name =~ {db_class_name_exp} 
    RETURN DISTINCT file, zk_couple, db_couple
    """
    zk_class_name_exp = "(?i)^org\.zkoss.*"
    class_name_exp = "(?i).*report.*"
    db_class_name_exp = "(?i)^java\.sql.*"
    yield query, dict(zk_class_name_exp=zk_class_name_exp, 
                      db_class_name_exp=db_class_name_exp, 
                      class_name_exp=class_name_exp)

In [5]:
def find_class_names():
    query = """
    MATCH (class:JavaClass)
    RETURN class.name as class_name
    """
    yield query, {}

In [7]:
def main(env):
    reporting_debt = env.neo4j.run_in_tx(find_reporting_debt)
    sonar_dups = env.sonar.get("api/duplications/show", dict(
            key="IdempiereKey:temp/org.adempiere.extend/src/test/functional/MBPGroupTest.java"))

In [9]:
import pandas as pd
import itertools


def commit_to_dict(c):
    d = dict(
        hexsha=c.hexsha,
        author_name = c.author.name,
        author_email = c.author.email,
        authored_date = c.authored_date,
#         stats_lines = c.stats.total['lines'],
#         stats_deletions = c.stats.total['deletions'],
#         stats_files = c.stats.total['files'],
#         stats_insertions = c.stats.total['insertions']
    )
    return d

def commit_parents(c):
    for p in c.parents:
        d = dict(
            commit=c.hexsha,
            parent_commit = p.hexsha
        )
        yield d
        
def export_commit_tree(env):
    commits = list(env.git.repo.iter_commits())
    commit_dicts = [commit_to_dict(c) for c in commits]
    cframe = pd.DataFrame(commit_dicts)
    cframe.to_csv(env.ws.abs_path_str("commits.csv"), index=False)
    cparents = itertools.chain.from_iterable((commit_parents(c) for c in commits))
    pframe= pd.DataFrame(list(cparents))
    pframe.to_csv(env.ws.abs_path_str("parents.csv"), index=False)

In [10]:
export_commit_tree(env)